In [1]:
import os
import sys
import numpy as np
import sympy as sp
import torch as pt
from torch.nn.functional import normalize
import scipy
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import logging
import sys
from torch.utils.data import TensorDataset, DataLoader
from torch.func import hessian, vmap
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")

sys.path.append(os.path.abspath('../'))

from src.useful_functions import *
from src.PWDs_module import generate_PWDistances_torch
from src.sqra_functions import*
from src.openmm_functions import*
from src.isokann_modules2 import*
# For reproducibility
np.random.seed(0)


# Read directory paths
read_dirs_paths('dir_paths_.txt', globals())



 
Created variables:
inp_dir = /scratch/htc/fsafarov/structures/8ef5_july_2025/8ef5/
dcd_dir = /scratch/htc/fsafarov/mOR_dcd_files/npat/
out_dir = /scratch/htc/fsafarov/ISOKANN_PINN/output/


In [2]:
psf_file = 'step5_assembly.psf'
crd_file = 'step5_assembly.crd'

system, psf, crd = setup_system(
                    inp_dir,
                    ligand_name = '7V7',
                    nbcutoff = 1.0,
                    psf_file=psf_file,
                    crd_file=crd_file  
                )


dcd_file = 'mOR_simulation_NPAT_CA_aligned.dcd'
dt=2.0
gamma=1.0
T=310.15
platform='CUDA'


forces = get_parameters(
                        system,
                        inp_dir,
                        dcd_file,
                        dcd_dir,
                        dt, #in femtoseconds
                        T,
                        gamma,
                        platform,
                        pdb_file='step5_assembly.pdb',
                        integrator_type='Langevin',
                        psf=psf,
                        get_potential_grad=True
                        
                      )


Number of atoms in the system: 118294


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.0; phase=0.0; per=3> with <DihedralType: k=0.12; phase=0.0; per=3>
  warnings.warn('Replacing dihedral %r with %r' %


dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [3]:
ca_indices = select(psf,selection="protein and name CA")
forces_ = np.linalg.norm(forces, axis=-1)

forces_ = forces_[:, ca_indices]


In [4]:
coords = get_parameters(
                        system,
                        inp_dir,
                        dcd_file,
                        dcd_dir,
                        dt, #in femtoseconds
                        T,
                        gamma,
                        platform,
                        pdb_file='step5_assembly.pdb',
                        integrator_type='Langevin',
                        psf=psf,
                        get_coords=True
                      )

dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [5]:
positions = np.linalg.norm(coords, axis=-1)
positions = positions[:, ca_indices]

In [6]:
forces_ = pt.Tensor(forces_)
positions = pt.Tensor(positions)

In [7]:
mean_forces = forces_.mean(dim=-1, keepdim=True)
std_forces = forces_.std(dim=-1, keepdim=True)
forces_ = abs(forces_ - mean_forces)/(std_forces + 1e-8)

In [8]:
mean_forces = forces_.mean(dim=-1, keepdim=True)
std_forces = forces_.std(dim=-1, keepdim=True)
forces_ = (forces_ - mean_forces)/(std_forces + 1e-8)

In [10]:
inp_dim = positions.shape[-1]
nodes = [inp_dim, 128, 64, 1] 
base_mlp = MLP(nodes, act_fun='sigmoid')

In [21]:
def laplacian_operator(model, x, h_val):
    """
    Computes the sum of second derivatives (Laplacian)
    using the Central Difference scheme.
    """
    input_dim = x.shape[0]
    chi_n = model(x.unsqueeze(0)) # Get value at current point [1, m]
    lap_chi = pt.zeros_like(chi_center)

    for i in range(input_dim):
        # Create a unit vector for the i-th dimension
        h_vec = pt.zeros_like(x)
        h_vec[i] = h_val
        
        # Perturb only the i-th dimension 
        chi_plus = model((x + h_vec).unsqueeze(0))
        chi_minus = model((x - h_vec).unsqueeze(0))
        
        # Add the 2nd derivative contribution for this dimension
        lap_chi += (chi_minus + chi_plus - 2 * chi_n) / (h_val**2)

    return lap_chi.squeeze(0) # Shape [m]

def generator_action(model, x, forces_fn, D, h):  
   
    grad_chi = nabla_chi(model, x)
    # print(f"Gradient shape: {grad_chi.shape}")
    # print(f"Chi function shape {chi.shape}")

    # None -> model, 0 -> batch dimensions of chi
    lap_chi = vmap(laplacian_operator, in_dims=(None, 0))(model, x, h)  # vmap over batch


    # print(f"Laplacian chi shape: {lap_chi.shape}")
    return chi, (-0.4*forces_fn * grad_chi.squeeze(-1)).sum(dim=1) + D * lap_chi 

In [22]:
L_chi = generator_action(base_mlp.to(device), positions[1, :].unsqueeze(0).to(device), forces_[1, :].unsqueeze(0).to(device), D=1, h=1)

ValueError: vmap(laplacian_operator, in_dims=(None, 0), ...)(<inputs>): in_dims is not compatible with the structure of `inputs`. in_dims has structure TreeSpec(tuple, None, [*,
  *]) but inputs has structure TreeSpec(tuple, None, [*,
  *,
  *]).